# Phase2 - RAG

If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [1]:
import json
import requests
import os
import openai
from enum import Enum
from pydantic import BaseModel
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

Found Azure OpenAI API Base Endpoint: https://cog-fkplarx5db7we.openai.azure.com/


This is the object model for receiving questions.

In [2]:

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_false = "true_false"
    popular_choice = "popular_choice"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Load custom data
Here's how you load custom data into an embedding model and how you use langchain to query it.


In [16]:
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Qdrant

import os
from openai import AzureOpenAI
from langchain.vectorstores import Qdrant


# use an embeddingsmodel to create embeddings
embeddings_model = AzureOpenAIEmbeddings(    
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
)

AZURE_OPENAI_VERSION="2024-02-01"

client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# 1. load data into qdrant db in aca or locally
# load your data
data_dir = "data/movies"
documents = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader).load()

# use local quadrant (for debugging)
qdrant = Qdrant.from_documents(
    documents,
    embeddings_model,
    location=":memory:",
    collection_name="movies",
)

# use quadrant in aca
# # --> aus Env, URL von Quadrant
# url = "http://localhost:6333"
# qdrant = Qdrant.from_documents(documents,embeddings_model,url=url,prefer_grpc=False,collection_name="my_movies",)


# 2. find relevant chunks from vector db
question = "Tell me about the latest Ant Man movie. When was it released?"
found_docs = qdrant.similarity_search(question)
found_docs_as_text = " "
for elem in enumerate(found_docs, start=1):    
    found_docs_as_text += " "+ elem[1].page_content

# 3. ask llm based on releveant chunks
system_prompt = "You are an assistant to the user, you are given some context below, please answer the query of the user with as detail as possible"

parameters = [system_prompt, ' Context:', found_docs_as_text , ' Question:', question]
joined_parameters = ''.join(parameters)

response = client.chat.completions.create(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : joined_parameters}],
    )

print (response.choices[0].message.content)


100%|██████████| 1/1 [00:00<00:00, 131.69it/s]

The latest Ant-Man movie, "Ant-Man and the Smorgs: Smorgmania," was released on February 15, 2026. It is an action-adventure science fiction film, featuring superhero partners Smorg Smorisson, Smorg McLean, and Jenny Smorgth. In this movie, they explore the Smorgs Realm and encounter strange new Smorgs, embarking on an adventurous journey that pushes their limits. The movie has gained popularity with an average vote of 9.5.


# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.



In [ ]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """
    #####
    # implement rag flow here
    ######
    
    print(answer)
    return answer

Use this snippet to try your method with several questions.

In [ ]:

ask = Ask(question="Tell me about the latest Ant Man movie. When was it released? ", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer)


Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase2 $AZURE_ENV_NAME
```
